In [ ]:
%pip install matplotlib
%pip install pymc
%pip install arviz
%pip install numpy
%pip install scikit-learn
%pip install pytensor

In [ ]:
import os
# os.environ["PYTENSOR_FLAGS"] = "optimizer=None,exception_verbosity=high"
os.environ["CXX"] = "/opt/homebrew/Cellar/gcc/14.2.0_1/bin/gcc-14"  # Using clang++ which has the standard C++ headers on macOS
import shutil
pytensor_cache_dir = os.path.expanduser("~/.pytensor/compiledir_macOS-15.2-arm64-arm-64bit-arm-3.11.0-64")
if os.path.exists(pytensor_cache_dir):
    shutil.rmtree(pytensor_cache_dir)
import pytensor
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
from sklearn.linear_model import LinearRegression
# 📌 Bayesian Linear Regression with PyMC3

# 📌 Generate Synthetic Data
np.random.seed(42)
X = np.linspace(0, 10, 50)[:, None]  # Feature (reshape to column vector)
true_slope, true_intercept = 2.5, 5
y = true_slope * X.flatten() + true_intercept + np.random.normal(scale=2, size=50)  # Add noise

# 📌 Train Ordinary Least Squares (OLS) Regression
ols_model = LinearRegression()
ols_model.fit(X, y)
ols_predictions = ols_model.predict(X)



with pm.Model() as bayesian_model:
    # Priors for the slope and intercept
    beta = pm.Normal("beta", mu=0, sigma=10)  # Prior for slope
    alpha = pm.Normal("alpha", mu=0, sigma=10)  # Prior for intercept
    sigma = pm.HalfNormal("sigma", sigma=1)  # Prior for noise

    # Likelihood (Observed Data)
    mu = alpha + beta * X.flatten()
    likelihood = pm.Normal("y", mu=mu, sigma=sigma, observed=y)

    # Posterior Sampling using NUTS sampler with modified settings
    trace = pm.sample(
        2000,
        tune=1000,
        return_inferencedata=True,
        cores=1,
        chains=2,
        progressbar=True,
        init='jitter+adapt_diag',  # Changed initialization method
        target_accept=0.95,
        random_seed=42
    )

# 📌 Posterior Mean Predictions
posterior_samples = az.extract(trace, num_samples=100)
bayesian_slope_samples = posterior_samples["beta"]
bayesian_intercept_samples = posterior_samples["alpha"]

# 📌 Compute Bayesian Regression Predictions (Mean of Posterior Samples)
bayesian_predictions = np.mean(bayesian_intercept_samples) + np.mean(bayesian_slope_samples) * X.flatten()

# 📌 Plot Comparison: OLS vs Bayesian Regression
plt.figure(figsize=(10, 5))
plt.scatter(X, y, label="Data", color="gray", alpha=0.6)
plt.plot(X, ols_predictions, label="OLS Regression", linestyle="dashed", color="blue")
plt.plot(X, bayesian_predictions, label="Bayesian Regression", color="red")
plt.fill_between(
    X.flatten(),
    np.percentile(bayesian_intercept_samples + bayesian_slope_samples * X.flatten(), 2.5, axis=0),
    np.percentile(bayesian_intercept_samples + bayesian_slope_samples * X.flatten(), 97.5, axis=0),
    color="red",
    alpha=0.2,
    label="Bayesian 95% Credible Interval"
)
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.title("Bayesian vs. OLS Regression")
plt.show()

# 📌 Print Bayesian Posterior Estimates
print("Bayesian Regression Posterior Mean Estimates:")
print(f"  - Slope (beta): {np.mean(bayesian_slope_samples):.3f}")
print(f"  - Intercept (alpha): {np.mean(bayesian_intercept_samples):.3f}")

# 📌 Arviz Summary of Bayesian Posterior
az.summary(trace, var_names=["alpha", "beta", "sigma"])


Initializing NUTS using jitter+adapt_diag...
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float64}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1916, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transfor


You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_a2jovgqu

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_22xqxa2r

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_pzhgmvme


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float64}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1916, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^


You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_g3x525ef

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_rqj_p86k

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_po8ig7k3


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float64}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1916, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float64}(1)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1916, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^^


You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_q2fj84b2

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_gy6ocekd

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_30dnwlwf


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float32}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1916, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1173, in constant_folding
    return unconditional_constant_folding.transform(fgraph, node)
           ^^^^^^^^^^^^^^^^^


You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_s8it8wh3

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_w3ggyw38

You can find the C code in this temporary file: /var/folders/cj/4gtf5x057tv46r9xh_3g0g340000gp/T/pytensor_compilation_error_39qfsayn


CompileError: Compilation failed (return status=1):
/usr/bin/clang++ -dynamiclib -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -fPIC -undefined dynamic_lookup -ld64 -I/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/include -I/Library/Frameworks/Python.framework/Versions/3.11/include/python3.11 -I/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytensor/link/c/c_code -L/Library/Frameworks/Python.framework/Versions/3.11/lib -fvisibility=hidden -o /Users/mcelikik/.pytensor/compiledir_macOS-15.2-arm64-arm-64bit-arm-3.11.0-64/tmp6n86vnrr/m2217b18d8d9783d8c4ae6857977a7e2bca8e2d5e4101c5fc9ffff0e09da7b747.so /Users/mcelikik/.pytensor/compiledir_macOS-15.2-arm64-arm-64bit-arm-3.11.0-64/tmp6n86vnrr/mod.cpp
/Users/mcelikik/.pytensor/compiledir_macOS-15.2-arm64